# LDA Model Results
<p>Here we will review teh results of the various ldam models that were generated. The goal is to idenity a model that has enough topics of interest while still providing significant deliniation between the topics.  Models are numbered Model 1 through Model 5, and descriptionsof each are included below.</p>

# Results
<p><b>Model 5</b> provides the best results and includes all reviews, we will be useing the results of Model 5 to Tag Reviews.</p>

## Import packages and establish chart output standards

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from gensim.models.ldamulticore import LdaMulticore
import itertools
from collections import Counter

___

### Pretty Printer Function

In [7]:
def print_topic_terms(model, num_topics=-1, num_words=10, unique=False):
    results = model.print_topics(num_topics=num_topics, num_words=num_words)
    if not unique:
        print('=============================== Terms Per Topic ===============================')
        for r in results:
            topic = r[0]
            term_list = r[1]

            term_list = term_list.split('"')[1::2]
            topic_terms = [term for term in term_list]
            print('{}\t{}'.format(topic, topic_terms))
    else:
        terms = [x[1] for x in results]
        term_lists = [x.split('"')[1::2] for x in terms]

        flatList = itertools.chain.from_iterable(term_lists)
        term_counts = Counter(flatList)

        # non_unique_terms = term_counts
        test = dict(term_counts)

        # extract terms that appear more than once
        non_unique_terms = [key for key, value in test.items() if value > 1]
        
        
        print('============================ Unique Terms Per Topic ===========================')
        for r in results:
            topic = r[0]
            term_list = r[1]

            term_list = term_list.split('"')[1::2]
            topic_terms = [term for term in term_list if term not in non_unique_terms]
            print('{}\t{}'.format(topic, topic_terms))
            

___

# Model 5 - All Reviews, All Tokens
<p>Looking at all Reviews, and limiting tokens to only nouns and verb tokens more common than the 10,000th most common noun or verb token.</p>

<ul>
    <li>Num Topics: 50</li>
    <li>Num Terms: 10</li>
    <li>Num Passes: 50</li>
    <li>Key Topics Identified: 1:Loyalty, 7:Wait Time, 8:Atmosphere, 9:Ordering, 13:Cleanliness, 15:Food Quality, 17:Customer Service, 26:Lunch Parking, 35:Price Value</li>
</ul>

In [8]:
model_05 = LdaMulticore.load('../models/ldam_all_restaurants_50_topics_10_terms_50_passes.model')

In [9]:
topics_of_interest = {'Wait Time': 7,
                      'Atmosphere': 8,
                      'Ordering': 9,
                      'Cleanliness' : 13,
                      'Customer Service': 17,
                      'Value': 35}

# Assigning Topic to Reviews

In [ ]:
import pandas as pd

In [ ]:
reviews = pd.read_csv('../clean_data/az_restaurant_reviews.csv', index_col=0)
reviews.head(3)

In [ ]:
biz = pd.read_csv('../clean_data/az_restaurant_business_clean.csv', index_col=0)
biz = biz.iloc[:,:9].copy()
biz.head(3)

In [ ]:
review_df = reviews.merge(biz[['name', 'business_id']], on='business_id', how='left')

In [ ]:
review_df.head(5)

In [ ]:
non_fast_food = reviews[reviews.is_fast_food == 0].copy()
non_fast_food.head(3)

In [ ]:
chipotle_ids = biz[biz.name == 'Chipotle Mexican Grill'].business_id.values

In [ ]:
chipotle_reviews = reviews[reviews.business_id.isin(chipotle_ids)].copy()
chipotle_reviews.head(3)

In [ ]:
def print_top_n_review_topics(model, review, n_topics=5, valid_topics = []):
    review_topic_categories = []
    for word in review.split(' '):
        try:
            r = model.get_term_topics(word_id = word)
            [review_topic_categories.append(x[0]) for x in r]
        except:
            pass
    
    print(review)
    print()

    top_n_topics = [x[0] for x in Counter(review_topic_categories).most_common(n_topics)]
    
    if len(valid_topics) > 0:
        # prune to only topics we care about
        topics = [topic for topic in top_n_topics if topic in valid_topics]
    else:
        topics = top_n_topics

    for n in topics:
        print('Topic {:d}'.format(n))
        print('\t{}\n'.format(model.print_topic(n)))

# Sample Bad Review topic Assignment

In [ ]:

sample_review = reviews[reviews.review_id == 'DbLUpPT61ykLTakknCF9CQ'].text.values[0]

print_top_n_review_topics(ldam_fast_food_501050, sample_review, n_topics=5, valid_topics = [12, 24, 27, 35, 43, 48])

In [ ]:
for rev in chipotle_reviews.text[:10]:
    print_top_n_review_topics(ldam_fast_food_501050, rev, n_topics=5, valid_topics = [12, 24, 27, 35, 43, 48])
    print('='*80)